In [1]:
import matplotlib.pyplot as plt

from Ballot import Ballot
from DefaultConfigOptions import *
from PartyPrimaryElection import PartyPrimaryElection
from ElectionResult import ElectionResult
from DistrictData import DistrictVotingRecord, DistrictData
from InstantRunoffElection import InstantRunoffElection
from HeadToHeadElection import HeadToHeadElection
from Population import Population
from NDPopulation import NDPopulation
from typing import List, Set, Callable
from Election import Election
from CandidateModel import CandidateModel
import tensorflow as tf
import tensorboard as tb
from Timings import Timings

In [2]:


class ElectionConstructor:
    def __init__(self, constructor: Callable[[List[Ballot], Set[Candidate]], Election], name: str):
        self.constructor = constructor
        self.name = name

    def run(self, ballots: List[Ballot], candidates: Set[Candidate]) -> ElectionResult:
        e = self.constructor(ballots, candidates)
        return e.result()


def construct_irv(ballots: List[Ballot], candidates: Set[Candidate]):
    return InstantRunoffElection(ballots, candidates)


def construct_h2h(ballots: List[Ballot], candidates: Set[Candidate]):
    return HeadToHeadElection(ballots, candidates)

In [3]:
class Sample:
    def __init__(self, opponents: List[Candidate], candidate: Candidate):
        self.opponents = opponents.copy()
        self.candidate = candidate

In [4]:
def create_population(ideology_dim: int) -> (CandidateModel, NDPopulation):
    population_means = np.zeros(shape=(ideology_dim,))
    population_stddev = np.ones(shape=(ideology_dim,)) * 40
    pop = NDPopulation(population_means, population_stddev, Independents)
    return pop

In [5]:
class Stats:
    def __init__(self):
        self.winner_distance = 0
        self.results = []


    def reset(self):
        self.winner_distance = 0
        self.results = []

    def update(self, winner: Candidate):
        self.winner_distance += winner.ideology.distance_from_o()
        self.results += [winner]


    def print(self, label: str):
        print("%15s %5d " %
               (label,
               len(self.results)), end="")

        print("%5.2f" % (self.winner_distance / len(self.results)))

In [6]:
def run_sample_election(process: ElectionConstructor, population: NDPopulation, timings: Timings):
    candidates = []
    for i in range(6):
        ideology = population.sample_voter().ideology
        c = Candidate("r-" + str(i), Independents, ideology, 0)
        candidates += [c]

    with timings.time_block("generate_voters"):
        voters = population.generate_voters(1000)

    with timings.time_block("compute_ballots"):
        ballots = [Ballot(v, candidates, default_election_config) for v in voters]

    with timings.time_block("run_process"):
        result = process.run(ballots, set(candidates))
    winner = result.winner()

    return winner, candidates

In [7]:
def run_process(process: ElectionConstructor, dim: int, samples: int):
    population = create_population(dim)
    timings = Timings()
    stats = Stats()
    for i in range(samples):
        winner, cc = run_sample_election(process, population, timings)
        stats.update(winner)

    return stats

In [15]:
class ProcessResult:
    def __init__(self, process: ElectionConstructor, dim: int, stats: Stats):
        self.process = process
        self.dim = dim
        self.stats = stats

    def print(self):
        self.stats.print("%12s - %dD" % (self.process.name, self.dim))

In [16]:
results = []
processes = [
    ElectionConstructor(constructor=construct_irv, name="Instant Runoff"),
    ElectionConstructor(constructor=construct_h2h, name="Head-to-Head")
]

for process in processes:
    for dim in [1,2,3,4]:
        stats = run_process(process, dim, 1000)

        results.append(ProcessResult(process, dim, stats))

for r in results:
    r.print()

Instant Runoff - 1D  1000 25.28
Instant Runoff - 2D  1000 35.25
Instant Runoff - 3D  1000 47.85
Instant Runoff - 4D  1000 58.31
Head-to-Head - 1D  1000  7.85
Head-to-Head - 2D  1000 20.66
Head-to-Head - 3D  1000 32.12
Head-to-Head - 4D  1000 42.90
